# Assignment 2

Niall Kelly, 24-743-601

Matej Mijatovic, 20-454-773

Christopher Yuan, 24-746-059

Kilian Picinali, 20-993-515

Yannick Secchi, 19-451-046

## 1. Linearization of Turnover

**(15 points)**

Turnover constraints are used to limit the amount of change in portfolio weights between periods, helping to manage transaction costs and maintain portfolio stability.

Your task is to implement a method `linearize_turnover_constraint` for the class `QuadraticProgram`, which modifies the quadratic programming problem to incorporate a linearized turnover constraint. This will involve updating the objective function coefficients, equality and inequality constraints, as well as the lower and upper bounds of the problem. 

Additionally, complete the example provided below to demonstrate that your method functions correctly.

In class, we discussed a solution that involved augmenting the dimensionality by a factor of three. Here, you are asked to implement an alternative method that involves a two-fold increase in dimensions. If you are unable to implement the two-fold method, you may proceed with the three-fold approach.

### Function Parameters:
- `x_init` (np.ndarray): The initial portfolio weights.
- `to_budget` (float, optional): The maximum allowable turnover. Defaults to `float('inf')`.

### Steps for Function Implementation:

As discussed in the lecture, introduce auxiliary variables and augment the matrices/vectors used for optimization.

- **Objective Function Coefficients**:  
  Pad the existing objective function coefficients (`P` and `q`) to accommodate the new variables introduced by the turnover constraint.  
  *Note*: "Padding" refers to adding extra elements (typically zeros) to an array or matrix to increase its size to a desired shape.

- **Equality Constraints**:  
  Pad the existing equality constraint matrix (`A`) to account for the new variables.

- **Inequality Constraints**:  
  Pad the existing inequality constraint matrix ('G') and vector ('h') and further add a new inequality constraint row to incorporate the turnover constraint.  

- **Lower and Upper Bounds**:  
  Pad the existing lower (`lb`) and upper (`ub`) bounds to accommodate the new variables.

- **Update Problem Data**:  
  Overwrite the original problem data in the `QuadraticProgram` class with the updated matrices and vectors to include the linearized turnover constraint.

In [1]:
# Import standard libraries
import types
import os
import sys

# Import third-party libraries
import numpy as np
import pandas as pd

# Import local modules
project_root = os.path.dirname(os.path.dirname(os.getcwd()))  # Change this path if needed
src_path = os.path.join(project_root, 'qpmwp-course\\src')
sys.path.append(project_root)
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.quadratic_program import QuadraticProgram
from helper_functions import load_data_msci

In [3]:
def linearize_turnover_constraint_2_fold(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem in the 2-fold case.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        # Dimensions
        n = len(self.problem_data.get('q'))
        m = 0 if self.problem_data.get('G') is None else self.problem_data.get('G').shape[0]

        # Update the coefficients of the objective function
        q = self.problem_data.get('q')
        q = np.pad(q, (0, n), 'constant', constant_values=0)

        P = self.problem_data.get('P')
        P = np.pad(P, ((0, n), (0, n)), 'constant', constant_values=0)

        # Update the equality constraints
        A = self.problem_data.get('A')
        r = A.shape[0]
        A = np.pad(A, ((0, 0), (0, n)), 'constant', constant_values=0)

        b = self.problem_data.get('b')

        # Update the inequality constraints
        G = self.problem_data.get('G')
        h = self.problem_data.get('h')

        if G is None:
            G = np.zeros((2*n + 1, 2*n))
            G[:n, :n] = np.eye(n)
            G[:n, n:] = -1 * np.eye(n)
            G[n:-1, :n] = -1 * np.eye(n)
            G[n:-1, n:] = -1 * np.eye(n)
            G[-1, n:] = np.ones(n)
            h = np.concatenate((x_init.to_numpy(), -1*x_init.to_numpy(), np.array([to_budget])))
        
        else:
            G = np.pad(G, ((0, 2*n+1), (0, n)), 'constant', constant_values=0)
            G[m:m+n, :n] = np.eye(n)
            G[m:m+n, n:] = -1 * np.eye(n)
            G[m+n:-1, :n] = -1 * np.eye(n)
            G[m+n:-1, n:] = -1 * np.eye(n)
            G[-1, n:] = np.ones(n)
            h = np.concatenate((h, x_init.to_numpy(), -1*x_init.to_numpy(), np.array([to_budget])))

        # Update lower and upper bounds
        lb = self.problem_data.get('lb')
        ub = self.problem_data.get('ub')

        lb = np.pad(lb, (0, n), 'constant', constant_values=0)
        lb[n:] = np.maximum(np.zeros(n), np.maximum(lb[:n] - x_init.to_numpy(), x_init.to_numpy() - ub)) # likely redundant

        ub = np.pad(ub, (0, n), 'constant', constant_values=0)
        ub[n:] = x_init + ub[:n]

        # Override the original matrices (notice: b does not change)
        self.update_problem_data({
            'P': P,
            'q': q,
            'G': G,
            'h': h,
            'A': A,
            'b': b,
            'lb': lb,
            'ub': ub
        })

        return None

In [4]:
def linearize_turnover_constraint_3_fold(self, x_init: np.ndarray, to_budget=float('inf')) -> None:
        '''
        Linearize the turnover constraint in the quadratic programming problem in the 3-fold case.

        This method modifies the quadratic programming problem to include a linearized turnover constraint.

        Parameters:
        -----------
        x_init : np.ndarray
            The initial portfolio weights.
        to_budget : float, optional
            The maximum allowable turnover. Defaults to float('inf').

        Notes:
        ------
        - The method updates the problem's objective function coefficients, inequality constraints,
        equality constraints, and bounds to account for the turnover constraint.
        - The original problem data is overridden with the updated matrices and vectors.

        Examples:
        ---------
        >>> qp = QuadraticProgram(P, q, G, h, A, b, lb, ub, solver='cvxopt')
        >>> qp.linearize_turnover_constraint(x_init=np.array([0.1, 0.2, 0.3]), to_budget=0.05)
        '''
        # Dimensions
        n = len(self.problem_data.get('q'))
        m = 0 if self.problem_data.get('G') is None else self.problem_data.get('G').shape[0]

        # Update the coefficients of the objective function
        q = self.problem_data.get('q')
        q = np.pad(q, (0, 2*n), 'constant', constant_values=0)

        P = self.problem_data.get('P')
        P = np.pad(P, ((0, 2*n), (0, 2*n)), 'constant', constant_values=0)

        # Update the equality constraints
        A = self.problem_data.get('A')
        r = A.shape[0]
        A = np.pad(A, ((0, n), (0, 2*n)), 'constant', constant_values=0)
        A[r:, :n] = np.eye(n)
        A[r:, n:2*n] = np.eye(n)*-1
        A[r:, 2*n:] = np.eye(n)

        b = self.problem_data.get('b')
        b = np.concatenate((np.array([b]), x_init.to_numpy()))

        # Update the inequality constraints
        G = self.problem_data.get('G')
        h = self.problem_data.get('h')

        if G is None:
            G = np.concatenate((np.zeros(n), np.ones(2*n)))
            h = np.array([to_budget])
        
        else:
            G = np.pad(G, ((0, 1), (0, 2*n)), 'constant', constant_values=0)
            G[n:, n:] = np.ones(2*n)
            h = np.concatenate(h, np.array([to_budget]))

        # Update lower and upper bounds
        lb = self.problem_data.get('lb')
        ub = self.problem_data.get('ub')

        lb = np.pad(lb, (0, 2*n), 'constant', constant_values=0)

        ub = np.pad(ub, (0, 2*n), 'constant', constant_values=0)
        ub[n:2*n] = 0.5*(to_budget + ub[:n] - x_init)
        ub[2*n:3*n] = 0.5*(to_budget - lb[:n] + x_init)

        # Override the original matrices (notice: b does not change)
        self.update_problem_data({
            'P': P,
            'q': q,
            'G': G,
            'h': h,
            'A': A,
            'b': b,
            'lb': lb,
            'ub': ub
        })

        return None

# Demo

##### Create P and q

In [5]:
# Load the msci country index data
N = 10
data = load_data_msci(path = '../data/', n = N)
X = data['return_series']

# Compute the vector of expected returns (mean returns)
q = ExpectedReturn(method='geometric').estimate(X=X, inplace=False)

# Compute the covariance matrix
P = Covariance(method='pearson').estimate(X=X, inplace=False)

# q, P

## 2-Fold Testing

##### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance. (2-fold Case)

In [6]:
# Instantiate the constraints with only the budget and long-only constraints
constraints = Constraints(ids = X.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
constraints.add_box(lower=0.0, upper=1.0)
GhAb = constraints.to_GhAb()

# Create a quadratic program and linearize the turnover constraint
qp_2_fold = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the 2-fold linearized turnover constraint method to the instance of class QuadraticProgram
qp_2_fold.linearize_turnover_constraint_2_fold = types.MethodType(linearize_turnover_constraint_2_fold, qp_2_fold)


##### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected. (2-fold Case)

In [7]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp_2_fold.linearize_turnover_constraint_2_fold(x_init=x_init, to_budget=0.5)

# Solve the problem
qp_2_fold.solve()

# Check the turnover
solution_2_fold = qp_2_fold.results.get('solution')
ids = constraints.ids
weights_2_fold = pd.Series(solution_2_fold.x[:len(ids)], index=ids)

print("Turnover:")
print(np.abs(weights_2_fold - x_init).sum())

Turnover:
0.4998179405734236


In [8]:
pd.concat([x_init.rename("initial_weights"), weights_2_fold.rename('optimized_2_fold_weights')], axis=1)

,initial_weights,optimized_2_fold_weights
AT,0.1,0.084243
AU,0.1,0.342180
BE,0.1,0.099985
CA,0.1,0.107682
CH,0.1,0.100026
DE,0.1,0.091709
DK,0.1,0.100021
ES,0.1,0.066553
FI,0.1,0.000056
FR,0.1,0.007545


## 3-Fold Testing

##### Create some constraints, instantiate an object of class QuadraticProgram, and add the method linearize_turnover_constraint to the instance. (3-fold Case)

In [9]:
# Create a quadratic program and linearize the turnover constraint
qp_3_fold = QuadraticProgram(
    P = P.to_numpy(),
    q = q.to_numpy() * 0,
    G = GhAb['G'],
    h = GhAb['h'],
    A = GhAb['A'],
    b = GhAb['b'],
    lb = constraints.box['lower'].to_numpy(),
    ub = constraints.box['upper'].to_numpy(),
    solver = 'cvxopt',
)

# Add the 2-fold linearized turnover constraint method to the instance of class QuadraticProgram
qp_3_fold.linearize_turnover_constraint_3_fold = types.MethodType(linearize_turnover_constraint_3_fold, qp_3_fold)

##### Add a turnover limit of 50%. Solve the problem and check whether the turnover constraint is respected. (3-fold Case)

In [10]:
# Prepare initial weights
x_init = pd.Series([1/X.shape[1]]*X.shape[1], index=X.columns)

# Add the linearized turnover constraint
qp_3_fold.linearize_turnover_constraint_3_fold(x_init=x_init, to_budget=0.5)

# Solve the problem
qp_3_fold.solve()

# Check the turnover
solution_3_fold = qp_3_fold.results.get('solution')
ids = constraints.ids
weights_3_fold = pd.Series(solution_3_fold.x[:len(ids)], index=ids)

print("Turnover:")
print(np.abs(weights_3_fold - x_init).sum())

Turnover:
0.4996230443248095


In [11]:
pd.concat([x_init.rename("initial_weights"), weights_3_fold.rename('optimized_3_fold_weights')], axis=1)

,initial_weights,optimized_3_fold_weights
AT,0.1,0.083472
AU,0.1,0.341682
BE,0.1,0.099985
CA,0.1,0.108051
CH,0.1,0.100045
DE,0.1,0.088974
DK,0.1,0.100035
ES,0.1,0.064275
FI,0.1,0.000628
FR,0.1,0.012855


## Comparing 3-Fold and 2-Fold

In [12]:
pd.concat([x_init.rename("initial_weights"), weights_2_fold.rename('optimized_2_fold_weights'), weights_3_fold.rename('optimized_3_fold_weights')], axis=1)

,initial_weights,optimized_2_fold_weights,optimized_3_fold_weights
AT,0.1,0.084243,0.083472
AU,0.1,0.342180,0.341682
BE,0.1,0.099985,0.099985
CA,0.1,0.107682,0.108051
CH,0.1,0.100026,0.100045
DE,0.1,0.091709,0.088974
DK,0.1,0.100021,0.100035
ES,0.1,0.066553,0.064275
FI,0.1,0.000056,0.000628
FR,0.1,0.007545,0.012855
